In [3]:
!pip install tensorflow

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [5]:
# Extracting music and features
'''
Dataset
We use GTZAN genre collection dataset for classification. 

The dataset consists of 10 genres i.e

Blues
Classical
Country
Disco
Hiphop
Jazz
Metal
Pop
Reggae
Rock
Each genre contains 100 songs. Total dataset: 1000 songs
'''

'\nDataset\nWe use GTZAN genre collection dataset for classification. \n\nThe dataset consists of 10 genres i.e\n\nBlues\nClassical\nCountry\nDisco\nHiphop\nJazz\nMetal\nPop\nReggae\nRock\nEach genre contains 100 songs. Total dataset: 1000 songs\n'

In [6]:
# Extracting the Spectrogram for every Audio

In [7]:

cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

'\ncmap = plt.get_cmap(\'inferno\')\n\nplt.figure(figsize=(10,10))\ngenres = \'blues classical country disco hiphop jazz metal pop reggae rock\'.split()\nfor g in genres:\n    pathlib.Path(f\'img_data/{g}\').mkdir(parents=True, exist_ok=True)     \n    for filename in os.listdir(f\'./MIR/genres/{g}\'):\n        songname = f\'./MIR/genres/{g}/{filename}\'\n        y, sr = librosa.load(songname, mono=True, duration=5)\n        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides=\'default\', mode=\'default\', scale=\'dB\');\n        plt.axis(\'off\');\n        plt.savefig(f\'img_data/{g}/{filename[:-3].replace(".", "")}.png\')\n        plt.clf()\n'

In [8]:
'''
Extracting features from Spectrogram
We will extract

Mel-frequency cepstral coefficients (MFCC)(20 in number)
Spectral Centroid,
Zero Crossing Rate
Chroma Frequencies
Spectral Roll-off.
'''

'\nExtracting features from Spectrogram\nWe will extract\n\nMel-frequency cepstral coefficients (MFCC)(20 in number)\nSpectral Centroid,\nZero Crossing Rate\nChroma Frequencies\nSpectral Roll-off.\n'

In [9]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [10]:
# Writing data to csv file

In [11]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())


"\nfile = open('data.csv', 'w', newline='')\nwith file:\n    writer = csv.writer(file)\n    writer.writerow(header)\ngenres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()\nfor g in genres:\n    for filename in os.listdir(f'./MIR/genres/{g}'):\n        songname = f'./MIR/genres/{g}/{filename}'\n        y, sr = librosa.load(songname, mono=True, duration=30)\n        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)\n        rmse = librosa.feature.rmse(y=y)\n        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)\n        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)\n        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)\n        zcr = librosa.feature.zero_crossing_rate(y)\n        mfcc = librosa.feature.mfcc(y=y, sr=sr)\n        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    \n        for e in mfcc:\n            to_append += f' {np.

In [12]:
# Analysing the Data in Pandas

In [13]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00060.au,0.430894,0.196222,1946.565652,1979.909934,3955.867746,0.097454,-67.770980,111.704184,-34.646105,...,12.295832,-12.477988,1.681278,-5.142068,4.644002,-6.919217,1.040718,-4.736871,-0.660037,blues
1,blues.00082.au,0.338896,0.251350,2141.461656,2168.015560,4627.997015,0.105151,-29.362093,108.667950,-25.573165,...,5.456504,-7.687713,7.410600,-11.319177,7.229288,-9.466552,1.930059,-6.328476,-1.304812,blues
2,blues.00030.au,0.263016,0.170081,1379.081742,2004.000850,3015.831764,0.039376,-206.987590,117.781468,23.256245,...,-8.015467,-17.616342,-8.138554,-8.646157,-15.538988,-15.331506,-9.664872,-10.103310,-17.835100,blues
3,blues.00007.au,0.307921,0.131785,1451.754147,1577.369917,2955.348796,0.061435,-179.395447,136.459244,-26.656359,...,-6.954827,-3.544535,-8.051242,-8.959537,-8.424337,-10.558885,-10.788159,-4.693749,-8.638613,blues
4,blues.00064.au,0.332480,0.117413,2553.232415,2280.128669,5148.102203,0.146852,-85.150250,88.806722,-16.322611,...,8.478453,-19.590226,6.413210,-13.779667,6.112037,-13.154644,3.933456,-7.615454,3.752626,blues


In [14]:
data.shape

(1000, 28)

In [15]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [16]:
# Encoding the Labels

In [17]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [18]:
# Scaling the Feature columns

In [19]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [20]:
# Dividing data into training and Testing set

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [22]:
len(y_train)

800

In [23]:
len(y_test)

200

In [24]:
X_train[10]

array([-0.47381459,  0.55637099,  1.03155087,  0.7354193 ,  0.74898954,
        1.00725641,  0.56680776, -1.12527311, -0.18683187, -1.21614088,
        0.1334671 , -1.20240975,  0.96203517, -1.22801527,  0.80756641,
       -0.98014561,  0.01410241, -1.30172897,  0.20846547, -1.28134964,
       -0.04602419, -0.97537067,  0.91973706,  0.56842323,  1.28358311,
        0.34215076])

In [25]:
# Classification with Keras
# Building our Network

In [27]:
from sklearn import svm
from sklearn import metrics

svb = svm.SVC(kernel='linear')
svb.fit(X=X_train, y=y_train)
results3 = svb.predict(X_test)

print("SVM for text data")
print(results3)

metrics.accuracy_score(y_test, results3)

SVM for text data
[5 0 6 2 6 8 6 3 7 7 0 8 2 4 2 3 7 6 5 8 3 5 9 6 2 2 1 5 4 1 5 4 3 5 4 3 0
 6 5 2 7 6 5 3 0 0 7 3 6 5 6 1 2 6 3 2 4 5 9 2 1 2 9 3 4 3 1 1 4 9 7 1 3 0
 5 3 5 9 2 4 8 1 1 0 5 3 7 5 2 7 5 3 7 0 1 0 9 2 0 2 0 7 2 1 3 8 3 7 4 6 8
 9 6 9 4 2 5 1 8 2 9 3 1 1 0 1 3 4 0 2 2 2 2 1 8 2 8 8 5 9 7 7 2 1 2 1 6 0
 0 5 3 4 2 2 3 1 6 5 5 2 3 3 9 4 7 3 0 5 0 0 5 6 0 0 9 3 6 8 2 4 0 3 8 2 2
 1 4 5 0 1 4 1 5 7 1 1 3 0 4 0]


0.615

In [28]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [29]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
800/800 [==============================] - 0s 271us/step - loss: 2.1515 - accuracy: 0.2600
Epoch 2/20
800/800 [==============================] - 0s 29us/step - loss: 1.8586 - accuracy: 0.3713
Epoch 3/20
800/800 [==============================] - 0s 32us/step - loss: 1.6442 - accuracy: 0.4137
Epoch 4/20
800/800 [==============================] - 0s 27us/step - loss: 1.4637 - accuracy: 0.4825
Epoch 5/20
800/800 [==============================] - 0s 26us/step - loss: 1.3108 - accuracy: 0.5238
Epoch 6/20
800/800 [==============================] - 0s 31us/step - loss: 1.1980 - accuracy: 0.5688
Epoch 7/20
800/800 [==============================] - 0s 26us/step - loss: 1.1137 - accuracy: 0.6125
Epoch 8/20
800/800 [==============================] - 0s 34us/step - loss: 1.0470 - accuracy: 0.6363
Epoch 9/20
800/800 [==============================] - 0s 30us/step - loss: 0.9864 - accuracy: 0.6725
Epoch 10/20
800/800 [==============================] - 0s 27us/step - loss: 0.9333 - accur

In [31]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 125us/step


In [32]:
print('test_acc: ',test_acc)

test_acc:  0.6650000214576721


In [33]:
# Tes accuracy is less than training data accuracy. This hints at Overfitting.

In [34]:
# Validating our approach
# Let's set apart 200 samples in our training data to use as a validation set:

In [35]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [36]:
# Now let's train our network for 20 epochs:

In [37]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 283us/step - loss: 2.2690 - accuracy: 0.1650 - val_loss: 2.1379 - val_accuracy: 0.3500
Epoch 2/30
600/600 [==============================] - 0s 47us/step - loss: 2.1133 - accuracy: 0.3650 - val_loss: 2.0114 - val_accuracy: 0.3300
Epoch 3/30
600/600 [==============================] - 0s 42us/step - loss: 1.9751 - accuracy: 0.3717 - val_loss: 1.8892 - val_accuracy: 0.3450
Epoch 4/30
600/600 [==============================] - 0s 42us/step - loss: 1.8323 - accuracy: 0.3767 - val_loss: 1.7906 - val_accuracy: 0.3500
Epoch 5/30
600/600 [==============================] - 0s 35us/step - loss: 1.6991 - accuracy: 0.4033 - val_loss: 1.6999 - val_accuracy: 0.4100
Epoch 6/30
600/600 [==============================] - 0s 33us/step - loss: 1.5703 - accuracy: 0.4483 - val_loss: 1.6167 - val_accuracy: 0.4500
Epoch 7/30
600/600 [==============================] - 0s 35us/step - loss: 1.4528 - accuracy: 0

In [38]:
results

[1.0994455432891845, 0.6499999761581421]

In [39]:
# Predictions on Test Data

In [40]:
predictions = model.predict(X_test)

In [41]:
predictions[0].shape

(10,)

In [42]:
np.sum(predictions[0])

1.0

In [43]:
np.argmax(predictions[0])

5